In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
torch.cuda.empty_cache()

In [15]:
class TransposerNN(nn.Module):

    def __init__(self, config_layer):
        """ Arguments:
                config_layer_list: a tensor used to configure network.
                    all input should have the same shape as the config_layer.
        """
        super(TransposerNN, self).__init__()
        self.d1, self.d2 = config_layer.size()
        # define 8-9 linear layers

        self.ll_1 = nn.Sequential(nn.Linear(self.d1 * self.d2, self.d2), nn.ReLU())
        self.ll_2 = nn.Sequential(nn.Linear(self.d2, self.d1), nn.ReLU())
        self.ll_3 = nn.Sequential(nn.Linear(self.d1, self.d1), nn.ReLU())
        self.ll_4 = nn.Sequential(nn.Linear(self.d1, self.d1), nn.ReLU())
        self.ll_5 = nn.Sequential(nn.Linear(self.d1, self.d1), nn.ReLU())
        self.ll_6 = nn.Sequential(nn.Linear(self.d1, self.d1 * self.d2), nn.ReLU())
        pass

    def forward(self, input_layer):
        """ Arguments:
                input_layer: a tensor to be passed through the network. Should have the shape of (self.d1, self.d2)
            Returns:
                A tensor of shape (self.d1, self.d2).
        """
        #print(input_layer.shape)
        l1_out = self.ll_1(input_layer)
        l2_out = self.ll_2(l1_out)
        l3_out = self.ll_3(l2_out)
        l4_out = self.ll_4(l3_out)
        l5_out = self.ll_5(l4_out)
        l6_out = self.ll_6(l5_out)

        return l6_out

In [3]:
data = pd.read_pickle('../test.pkl')

input_data = []
target_data = []
for i in range(len(data)):
    if i % 2 == 0:
        input_data.append(data[i][0])
    else:
        target_data.append(data[i][0])

In [4]:
print(len(input_data[0]))
print(np.array(input_data[0]).shape)

150
(150, 128, 2048)


In [16]:
myNet = TransposerNN(input_data[0][0]).cuda()

In [17]:
print(input_data[0][0].shape)
out = myNet(torch.flatten(input_data[0][0]).cuda())
out = out.view(input_data[0][0].shape)
print(out.shape)
print(out)

torch.Size([128, 2048])
torch.Size([128, 2048])
tensor([[0.0000, 0.0964, 0.0000,  ..., 0.0000, 0.0616, 0.0467],
        [0.0000, 0.0551, 0.1295,  ..., 0.0000, 0.0611, 0.0000],
        [0.0000, 0.1134, 0.0000,  ..., 0.0159, 0.0523, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0118,  ..., 0.0546, 0.0000, 0.0759],
        [0.0000, 0.0000, 0.0678,  ..., 0.1164, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0889, 0.0132, 0.0000]],
       device='cuda:0', grad_fn=<ViewBackward>)


In [ ]:
# data is in the form of:
# datum = [
    #     big_array,  # ORIG ACTIV
    #     orig_classification,  # ORIG LABEL | [1,0] TRUE [0,1] FALSE
    #     None,  # this no longer used
    #     model_name,  # LANG MODEL: model_name an abstraction for 'gpt2'
    #     {'num_gpt2_iters': num_gpt2_iters_run, \
    #         'orig_tokens': orig_tokens, \
    #         'gpt2_generated_tokens': gpt2_generated_tokens},  # META DATA
    #     orig_text,  # ORIG TEXT (or what we're deeming 'originial text')
    #     None,  # PRED TEXT this literally won't exist until we have an NPI
    #     TARG[0],  # TARG TEXT
    #     gpt2_generated_text  # GPT2 TEXT: just generated right here by the GPT2 :D :D
    # ]
print(data[0][0][0].size())